# Opening Lead

This is a tutorial for how to use the opening lead engine.

In [1]:
import os
import sys
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

from nn.models import Models
from bots import BotLead
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies

INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding-10724000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo-6131000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


In [3]:
# both vulnerable. you are sitting North as dealer and you hold
hand = 'KQ854.854.JT96.8'

# the auction goes:
auction = ["PAD_START", "PAD_START", "1C", "PASS", "1H", "PASS", "3S", "PASS", "4D", "PASS", "4H", "PASS", "4N", "PASS", "5H", "PASS", "6H", "PASS", "PASS", "PASS"]

# what to lead?

lead_bot = BotLead([True, True], hand, models, sampler, False)
lead = lead_bot.find_opening_lead(auction)

Loaded lib dds.dll


In [4]:
lead.card

C8

seems like the engine chose to lead the ace of diamonds

the other options it considered were: a small spade and a small club

In [5]:
lead.to_dict()['candidates']

[{'card': 'C8',
  'insta_score': 0.23,
  'expected_tricks_sd': 12.5,
  'p_make_contract': 0.1},
 {'card': 'DJ',
  'insta_score': 0.247,
  'expected_tricks_sd': 12.5,
  'p_make_contract': 0.0},
 {'card': 'SK',
  'insta_score': 0.407,
  'expected_tricks_sd': 12.7,
  'p_make_contract': 0.0}]

in the above output:
- `insta_score` reflects the preference of the neural network
- `expected_tricks` how many tricks declarer is expected to take on this lead
- `p_make_contract` is the probability of the contract making on this lead

the probability of making and the expected tricks are computed on samples which are consistent with the auction. the samples are estimated single dummy using a neural network (more on this in another tutorial). we could also solve the samples double dummy, but that would be quite a bit slower.

In [6]:
# each row is one sample board
# the hands are in the order: LHO, Partner, RHO. Your cards are not shown as they are fixed/

lead.samples

['.AJxx.K8xx.AKJxx JT9x.T9.Qxx.Q9xx Axxx.KQxx.Ax.Txx 0.31481',
 '.9xxx.AKQx.AKJxx xxxx.Jx.8xx.QT9x AJT9.AKQT.xx.xxx 0.30424',
 '.KJT9.Kxx.AKQTxx T9xx.xx.A8xx.9xx AJxx.AQxx.Qx.Jxx 0.29240',
 'x.AJxx.KQx.AKJxx T9xx.Tx.xxx.QT9x AJx.KQ9x.A8x.xxx 0.29125',
 '9.ATxx.KQx.AKQJx JTxx.Jx.8xx.T9xx Axx.KQ9x.Axx.xxx 0.27554',
 'x.AJTx.KQ8.AKJxx JT9.xx.xxxx.9xxx Axxx.KQ9x.Ax.QTx 0.26551',
 '.AKJxx.Qx.KQJxxx JT9xx.x.8xxx.A9x Axx.QT9x.AKx.Txx 0.25891',
 '.J9xx.AKxx.AKJTx xxxx.Kx.8xx.9xxx AJT9.AQTx.Qx.Qxx 0.25681',
 '.AJxx.KQxx.AKQxx JTxxx.QT.8xx.Jxx A9x.K9xx.Ax.T9xx 0.24686',
 '.ATxx.Qxx.AKQJTx T9xxx.Q.A8xx.9xx AJx.KJ9xx.Kx.xxx 0.23653']